In [6]:
import numpy as np
from astropy.table import Table, Column, join
from astropy.coordinates import SkyCoord
from astropy.table import Column
from numpy import *
import matplotlib.pyplot as plt
%matplotlib inline
from astropy.wcs import WCS
from astropy.io import fits
from matplotlib.colors import LogNorm
from astropy.utils.data import download_file
import warnings

catalog_path='/Users/EvanB.LAPTOP-8CHCIOMA/Downloads/'

#A100-NSA Catalog
myTab=Table.read(catalog_path+'a100-nsa-for-adap.fits')

In [22]:
# R-band Magnitude Limit
Mrmax = -18.2
Mr = myTab['ABSMAG'][:,4]
Mrflag = Mr < Mrmax

# Velocity input
vinput = int(input())
v = myTab['Vhelio']
vflag = (v > -1*vinput) & (v < vinput)

cut = Mrflag & vflag

cuttab = myTab[cut]

 1500


In [23]:
print(len(myTab['ABSMAG'][:,4]))
print(len(cuttab['ABSMAG'][:,4]))

39098
10692


# Computing 5th Nearest Neighbor

In [29]:
ra = np.array(cuttab['RA'])
dec = np.array(cuttab['DEC'])
fifthnn = np.zeros(len(ra))

#for i in Tab:
    #d1 = ra[i]-ra[i+4]
    #d2 = dec[i]-dec[i+5]
    #if abs(ra) <= 5 & abs(dec) <= 5:
        
